In [ ]:
import gymnasium
import matplotlib
import numpy as np
import sys
import pandas as pd

from collections import defaultdict
from environment import PowerGridEnv
from scenarios_utils import get_scenario

In [ ]:
df_eco = pd.read_csv("eco2mix-national-tr.csv",delimiter=";")

In [ ]:
scenario,cons_max,prod_max = get_scenario(df_eco)

In [ ]:
#Scenario à la main

# scenario = np.array([(50,0),(110,0),(150,0),(220,0),(220,0),(210,30),
#                     (230,50),(350,100),(400,120),(430,150),(400,200),(400,200),
#                     (450,250),(350,170),(320,80),(320,60),(350,50),(230,30),
#                     (200,30),(210,10),(210,10),(220,0),(220,0),(210,0)])
# cons_max=scenario[:,0].max()
# prod_max = scenario[:,1].max()

In [ ]:
env = PowerGridEnv(scenario,battery_max=350)

In [ ]:
def get_indice(observations):
    state_indices = [0,0,0,0]
    bins = [10,10,3,24]
    for i,o in enumerate(observations) :
        state_indices[i] = np.floor(o * (bins[i] - 1)).astype(int)
        state_indices[i] = np.clip(state_indices[i], 0, bins[i] - 1)
    return tuple(state_indices)

In [ ]:
def mc_prediction(policy, env, num_episodes, gamma=1.0):
    """
    Monte Carlo prediction algorithm. Calculates the value function
    for a given policy using sampling.
    
    Args:
        policy: A function that maps an observation to action probabilities.
        env: OpenAI gym environment.
        num_episodes: Number of episodes to sample.
        discount_factor: Gamma discount factor.
    
    Returns:
        A dictionary that maps from state -> value.
        The state is a tuple and the value is a float.
    """

    # Keeps track of sum and count of returns for each state
    # to calculate an average. We could use an array to save all
    # returns (like in the book) but that's memory inefficient.
    returns_sum = defaultdict(lambda: np.zeros(5))
    returns_count = defaultdict(lambda: np.zeros(5))
    # The final value function
    Q = defaultdict(lambda: np.zeros(5))
    count = 0
    while True:
        states=[]
        done = False
        observations = env.reset()
        while not done :
            action = policy(observations)
            next_observations,reward,done = env.step(action) 
            states.append((get_indice(observations),action,reward))
            observations=next_observations
        G = 0
        for s,a,r in reversed(states):
            G = r + gamma * G  # Le retour G_t = r_t + gamma * G_{t+1}
                
            returns_sum[s][a] += G
            returns_count[s][a] += 1
            Q[s][a] = returns_sum[s][a] / returns_count[s][a]
        count+=1
        

            
        if count == num_episodes: break
    return Q  

In [ ]:
def random(observations):
    action = np.random.randint(5)
    return action 
def heuristic_policy(observations):
    psol = observations[1]
    battery = observations[2]
    hour = observations[3]

    # 1. Gestion de la CHARGE (Priorité au surplus solaire)
    # On teste d'abord le seuil le plus haut
    if psol > 0.8:
        action = 0  # Charge 100%
        print("Solaire fort : Charge 100%")
    elif psol > 0.6:
        action = 1  # Charge 50%
        print("Solaire moyen : Charge 50%")

    # 2. Gestion de la DÉCHARGE (Priorité au manque de soleil ou fin de journée)
    elif psol < 0.1 and battery == 1:
        action = 4  # Décharge 100%
        print("Batterie pleine & pas de soleil : Décharge 100%")
    elif psol < 0.3 and battery > 0.5:
        action = 3  # Décharge 50%
        print("Solaire faible : Décharge 50%")
    elif hour > 22 and battery > 0.5:
        action = 3  # Décharge 50%
        print("Fin de journée : On utilise la batterie")# 3. PAR DÉFAUT

    else:
        action = 2  # Idle (Rien ne se passe)
        print("Rien à faire (Idle)")

    return action

In [ ]:
Q_10k = mc_prediction(heuristic_policy, env, num_episodes=1)

In [ ]:
Q_10k